In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import shap
import lime
import lime.lime_tabular

In [10]:
# Load the datasets
fraud_data = pd.read_csv('../data/Fraud_Data.csv')
ip_data = pd.read_csv('../data/IpAddress_to_Country.csv')
creditcard_data = pd.read_csv('../data/creditcard.csv')

In [8]:
fraud_data

,user_id,purchase_value,device_id,source,browser,sex,age,ip_address,class,signup_hour,signup_day,purchase_hour,purchase_day
0,22058,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,22,1,2,5
1,333320,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,20,6,1,0
2,1359,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,18,3,18,3
3,150084,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,21,1,13,0
4,221365,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,7,1,18,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,43,XPSKTWGPWINLR,SEO,Chrome,M,28,3.451155e+09,1,3,1,0,6
151108,274471,35,LYSFABUCPCGBA,SEO,Safari,M,32,2.439047e+09,0,17,4,12,1
151109,368416,40,MEQHCSJUBRBFE,SEO,IE,F,26,2.748471e+09,0,23,1,7,2
151110,207709,46,CMCXFGRHYSTVJ,SEO,Chrome,M,37,3.601175e+09,0,20,3,9,0


In [11]:
# Convert datetime strings to datetime objects
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])
